## FINAL PROJECT 
AMPARO ALÍAS CUESTA

*--------------------------------------------------------------------------------------------------------------------*

### Required imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from tensorflow.keras.models import load_model

pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings("ignore")

### Load all data 

NUMERICAL:

In [31]:
X_trainNum = np.load('data/numerical/X_train.npy')
X_testNum = np.load('data/numerical/X_test.npy')

y_train = pd.read_csv('data/numerical/y_train.csv', sep = ';',index_col= 0, header = None)
y_test = pd.read_csv('data/numerical/y_test.csv', sep = ';',index_col = 0, header = None)

IMAGES:

In [34]:
loaded_images = np.load('images_branch/images.npy')

In [35]:
train_index = y_train.index.to_list()
test_index = y_test.index.to_list()

In [36]:
X_trainImg = loaded_images[train_index]
X_testImg = loaded_images[test_index]

In [48]:
X_trainImg = X_trainImg / 255.0
X_testImg = X_testImg / 255.0

OBJECTIVE VARIABLE:

In [37]:
y_train = y_train.values
y_test = y_test.values

In [39]:
maxPrice = y_train.max()
y_train = y_train / maxPrice
y_test = y_test / maxPrice

### MLP 

In [40]:
# returns a compiled model
# identical to the previous one
mlp = load_model('numerical_branch/MLP_model_reg.h5')

Evaluate the model and show the summary 

In [41]:
preds = mlp.evaluate(x = X_testNum, y = y_test)
print()
print("Loss = " + str(preds))

10/10 [==============================] - 0s 3ms/step - loss: 0.0797

Loss = 0.07971682399511337


### CNN 

In [44]:
# returns a compiled model
# identical to the previous one
cnn = load_model('images_branch/CNN_model_reg.h5')

Evaluate the model and show the summary 

In [49]:
preds = cnn.evaluate(x = X_testImg, y = y_test)
print()
print("Loss = " + str(preds))

10/10 [==============================] - 2s 214ms/step - loss: 0.1360

Loss = 0.13603226840496063


### Multiple inputs training

In [58]:
from keras.layers import concatenate, Dense
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Activation, Dropout, Dense, Flatten, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

In [56]:
combinedInput = concatenate([mlp.output, cnn.output])

In [60]:
# our final FC layer head will have two dense layers, the final one
# being our regression head
x = Dense(4, activation="relu", name = 'dense1')(combinedInput)
x = Dense(1, activation="linear", name = 'dense2')(x)

In [61]:
# our final model will accept categorical/numerical data on the MLP
# input and images on the CNN input, outputting a single value (the
# predicted price of the house)
model = Model(inputs=[mlp.input, cnn.input], outputs=x)

In [69]:
opt = Adam(lr=0.001, decay=1e-3/ 200)

model.compile(loss="mean_absolute_error", optimizer=opt)

# train the model
print("[INFO] training model...")
# simple early stopping
es = EarlyStopping(monitor='val_loss', verbose=1)

model.fit(
    x = [X_trainNum,X_trainImg], 
    y = y_train, 
    epochs=100, 
    validation_data=([X_testNum,X_testImg], y_test), 
    batch_size=16,
    callbacks=[es]
)

[INFO] training model...
Epoch 1/100
125/125 [==============================] - 58s 468ms/step - loss: 0.0781 - val_loss: 0.0715
Epoch 2/100
125/125 [==============================] - 57s 454ms/step - loss: 0.0787 - val_loss: 0.0717
Epoch 00002: early stopping
